In [1]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

# 벡터 DB : Chroma vs Pinecone

- Chroma : 인메모리DB, 로컬메모리 DB
- Pinecone : 클라우드 vector DB
  - [Pinecone console 에 api key 생성 -> .env (PINECONE_API_KEY)]
- pip install langchain-pinecone
- pip install pinecone-client pinecone-text


In [2]:
# %pip install langchain-pinecone pinecone-client pinecone-text

# 1. Knowledge Base 구성을 위한 데이터 생성


In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader("./tax_docs/with_markdown.docx")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [4]:
len(document_list)

225

In [5]:
# embedding : upstage embedding-query
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large",
    # model="embedding-query"
)

In [6]:
# pinecon vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()

# 데어터를 처음 업로드할 떄
index_name = "tax-index-markdown"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embeddings,
#     index_name=index_name,
# )

/Users/teamkim/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 업로드한 벡터DB 가져올 떄
database = PineconeVectorStore(
    embedding=embeddings, # 질문에 입배딩하여 유사도 검색
    index_name=index_name,
)

# 2. 답변 생성선 Retrieval 확인


In [8]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retriever = database.as_retriever(search_kwargs={"k": 3})
retriever.invoke(query)

[Document(id='56cc5615-efcb-4d6a-93a8-dff9481b1134', metadata={'source': './tax_docs/with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에

# 3. 제동되는 prompt 를 활용하여 답변 생성


In [9]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1-nano")

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=retriever, # database.as_retriver()
)

In [10]:
query = "연봉 5천만원인 거주자의 소득세는 얼마인가요?"

ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 해당 과세표준이 5,000만원 이하이므로, 세율은 84만원 + (연봉 초과 금액의 15%)의 방식으로 계산됩니다. 따라서, 연봉 5천만원인 경우, 과세표준이 5,000만원이 되어 세율은 84만원 + (0원)으로 총 84만원입니다. 즉, 소득세는 약 84만원입니다.'}

In [13]:
# 사람을 나타내는 표현 -> 거주자로 변경
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전 : {dictionary}
질문 : {{question}}""")

In [14]:
dictionary_chain = prompt | llm | StrOutputParser()
dictionary_chain.invoke({"question":query})

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [15]:
new_chain = {"query":dictionary_chain} | qa_chain

In [16]:
new_chain.invoke({"question":query})

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 약 624만원입니다. 이는 5,000만원 과세표준에 해당하는 구간 세율(24%)을 적용한 결과입니다. 실제 세액은 공제 등 기타 요소에 따라 다를 수 있습니다.'}